In [27]:
import os, sys
os.chdir('/home/pedro/SourceCode/map_matching')
import pandas as pd
from map_matching import *
import numpy as np
from map_matching.aequilibrae import Graph, PathResults, path_computation
from geopy.distance import vincenty as gc
from datetime import timedelta
from rtree import indexdef InterpolateTS(arrives_tstamps, leaves_tstamps, pathnodes, all_links,  network):
    graph = network.graph
    consistent = False
    while not consistent:
        consistent = True
        for i in range(len(pathnodes)-1):
            o = pathnodes[i]
            if o in arrives_tstamps.keys():
                for j in xrange(i + 1, len(pathnodes)):
                    d = pathnodes[j]
                    if d in arrives_tstamps.keys():
                        if arrives_tstamps[o] > arrives_tstamps[d]:
                            if i > 0:
                                leaves_tstamps.pop(o, None)
                                arrives_tstamps.pop(o, None)
                            if d != pathnodes[-1]:
                                arrives_tstamps.pop(d, None)
                                leaves_tstamps.pop(d, None)
                            consistent = False
                            break
                if not consistent:
                    break
    i = 0
    while i < len(pathnodes)-2:
        j = i + 1
        mp = network.orig_cost[all_links[j - 1]]
        while pathnodes[j] not in arrives_tstamps.keys():
            mp += network.orig_cost[all_links[j]]
            j += 1
        
        if j > i + 1:  # Means we have at least one node in the path that does not have any path written to it
            time_diff = (arrives_tstamps[pathnodes[j]] - leaves_tstamps[pathnodes[i]]).total_seconds()
            if time_diff < 0:
                del arrives_tstamps
                del leaves_tstamps
                break
            mp2 = 0
            for k in xrange(i + 1, j):
                mp2 += network.orig_cost[all_links[k - 1]]
                j_time = leaves_tstamps[pathnodes[i]] + timedelta(seconds=time_diff*mp2/mp)
                arrives_tstamps[pathnodes[k]] = j_time
                leaves_tstamps[pathnodes[k]] = j_time
        i += 1




In [28]:
out_folder = load_parameters('output_folder')

trips = Trip()

# data quality parameters
p = load_parameters('data quality')
trips.set_data_quality_parameters(p)

trips.set_stop_algorithm('Maximum space')
p = load_parameters('stops parameters')[trips.stops_algorithm]
trips.set_stops_parameters(p)

# Loading the GPS DATA
df = pd.read_csv('example_data/example_data_generic_gps_data.csv')
df['datetime']= df.Date.astype(str) + ' ' + df.time.astype(str)
df.datetime = pd.to_datetime(df.datetime)

# Data source specific
field_dict = {"latitude": "LATITUDE",
              "longitude": "LONGITUDE",
              "timestamp": "datetime"}

trips.populate_with_dataframe(df, field_dict)

In [29]:
# Loading the NETWORK
net = Network(out_folder)
par = load_parameters('geoprocessing parameters')
net.set_geometry_parameters(par)

net.load_nodes('example_data/FAF_Nodes.shp', 'ID')

p = load_parameters('network file fields')
net.load_network('example_data/FAF_Network.shp', p)

Creating nodes spatial index
     NODES spatial index created successfully
Creating graph from shapefile
Some skim field with wrong type. Converting to float64
Creating network spatial index
      Link 39659 could not have its buffer computed
      Link 40695 could not have its buffer computed
      Link 42165 could not have its buffer computed
      Link 44233 could not have its buffer computed
      Link 45159 could not have its buffer computed
      Link 119094 could not have its buffer computed
     LINKS spatial index created successfully


In [30]:
find_stops(trips)
find_network_links(trips, net)

In [31]:
trip = trips
network = net

In [49]:
vehicle_trace = trip.gps_trace
stops = trip.stops
a = pd.Series(trip.graph_links).unique()

network.reset_costs()
network.graph.cost[a[:]] /= 10

# # We select the nodes that are part of the signalized links
nodes_a = network.graph.graph['a_node'][network.graph.ids[a]]
nodes_b = network.graph.graph['b_node'][network.graph.ids[a]]
all_nodes = np.hstack((nodes_a, nodes_b))
all_nodes = np.bincount(all_nodes)

# Correlate the stop sequence with the nodes signalized to find the node from which we will trace the paths from/to
stop_sequence = []
for i in stops.index:
    x = stops.longitude[i]
    y = stops.latitude[i]
    t = stops.stop_time[i]
    dur = stops.duration[i]
    closest_nodes = list(network.idx_nodes.nearest((y, x, y, x), 5000))

    found_node = False
    for j in closest_nodes:
        if j < all_nodes.shape[0]:
            if all_nodes[j] > 0:
                found_node = True
                same = False
                if stop_sequence:
                    if stop_sequence[-1][0] == j:
                        same = True
                if not same:
                    stop_sequence.append((j, t, dur))  # The time stamp and duration will be needed to setup
                    # the time stamps for all the nodes along the path
                break
    if not found_node:
        stop_sequence.append((closest_nodes[0], t, dur))

results = PathResults()
results.prepare(network.graph)
#Now we need to compute the paths with the sequence of stops
all_links = []
all_nodes = []
all_arrives_tstamps = []
all_leaves_tstamps = []

x = np.zeros(1, np.int64)
x[0] = -1

In [50]:
def InterpolateTS(arrives_tstamps, leaves_tstamps, pathnodes, all_links,  network):
    graph = network.graph
    consistent = False
    while not consistent:
        consistent = True
        for i in range(len(pathnodes)-1):
            o = pathnodes[i]
            if o in arrives_tstamps.keys():
                for j in xrange(i + 1, len(pathnodes)):
                    d = pathnodes[j]
                    if d in arrives_tstamps.keys():
                        if arrives_tstamps[o] > arrives_tstamps[d]:
                            if i > 0:
                                leaves_tstamps.pop(o, None)
                                arrives_tstamps.pop(o, None)
                            if d != pathnodes[-1]:
                                arrives_tstamps.pop(d, None)
                                leaves_tstamps.pop(d, None)
                            consistent = False
                            break
                if not consistent:
                    break
    i = 0
    while i < len(pathnodes)-2:
        j = i + 1
        mp = network.orig_cost[all_links[j - 1]]
        while pathnodes[j] not in arrives_tstamps.keys():
            mp += network.orig_cost[all_links[j]]
            j += 1
        
        if j > i + 1:  # Means we have at least one node in the path that does not have any path written to it
            time_diff = (arrives_tstamps[pathnodes[j]] - leaves_tstamps[pathnodes[i]]).total_seconds()
            if time_diff < 0:
                del arrives_tstamps
                del leaves_tstamps
                break
            mp2 = 0
            for k in xrange(i + 1, j):
                mp2 += network.orig_cost[all_links[k - 1]]
                j_time = leaves_tstamps[pathnodes[i]] + timedelta(seconds=time_diff*mp2/mp)
                arrives_tstamps[pathnodes[k]] = j_time
                leaves_tstamps[pathnodes[k]] = j_time
        i += 1




In [62]:
for i in range(len(stop_sequence) - 1):
    distances = {}
    origin = stop_sequence[i][0]
    destination = stop_sequence[i + 1][0]
    if origin != destination:
        arrives_tstamps = {}
        leaves_tstamps = {}
        path_computation(origin, destination, network.graph, results)
        if results.path is not None:
            all_links.append(results.path.copy())
            all_links.append(x)
            all_nodes = all_nodes + list(results.path_nodes)

            #Now we build a spatial index with the subset of pings in the truck trace between the origin and destination
            ping_subset = vehicle_trace[(vehicle_trace['timestamp'] > stop_sequence[i][1] + timedelta(seconds=stop_sequence[i][2])) & (vehicle_trace['timestamp'] < stop_sequence[i + 1][1])]

            temp_idx = index.Index()
            for q in all_nodes:
                x = network.nodes.X[q]
                y = network.nodes.Y[q]
                bbox=(x, y, x, y)
                temp_idx.insert(q, bbox)

            for k in ping_subset.index:
                x = vehicle_trace.longitude[k]
                y = vehicle_trace.latitude[k]
                near = list(temp_idx.nearest((x, y, x, y), 1))
                if len(near) > 0:
                    q = near[0]
                    x2 = network.nodes.X[q]
                    y2 = network.nodes.Y[q]
                    d = gc((y, x), (y2, x2))
                    if q not in distances:
                        leaves_tstamps[q] = vehicle_trace.timestamp[k]
                        arrives_tstamps[q] = vehicle_trace.timestamp[k]
                        distances[q] = d
                    else:
                        if d < distances[q]:
                            leaves_tstamps[q] = vehicle_trace.timestamp[k]
                            arrives_tstamps[q] = vehicle_trace.timestamp[k]
                            distances[q] = d

            # Time stamps for the origin and destination of the path
            leaves_tstamps[origin] = stop_sequence[i][1] + timedelta(seconds=stop_sequence[i][2])
            arrives_tstamps[destination] = stop_sequence[i + 1][1]
            print arrives_tstamps
            InterpolateTS(arrives_tstamps, leaves_tstamps, results.path_nodes, results.path.copy(), network)
            print arrives_tstamps
            # For completeness
            if 0 < stop_sequence[i + 1][2] < 100000:
                leaves_tstamps[destination] = arrives_tstamps[destination] + timedelta(seconds=stop_sequence[i + 1][2])
            else:
                leaves_tstamps[destination] = np.nan

            if 0 < stop_sequence[i][2] < 100000:
                arrives_tstamps[origin] = stop_sequence[i][1]
            else:
                arrives_tstamps[origin] = np.nan

            q1 = []
            q2 = []
            for k in all_nodes[:-1]:
                if k in arrives_tstamps.keys():
                    q1.append(arrives_tstamps[k])
                else:
                    print "No node", k, 'in the arrives stamps'
                if k in leaves_tstamps.keys():
                    q2.append(leaves_tstamps[k])
                else:
                    print "No node", k, 'in the leaves stamps'
            all_arrives_tstamps.append(q1)
            all_leaves_tstamps.append(q2)
        results.reset()

{16128: Timestamp('2015-06-19 08:00:00'), 18401: Timestamp('2015-06-20 01:00:00'), 16130: Timestamp('2015-06-19 13:00:00'), 18404: Timestamp('2015-06-20 00:00:00'), 18310: Timestamp('2015-06-19 22:00:00'), 16134: Timestamp('2015-06-19 16:00:00'), 15687: Timestamp('2015-06-19 17:00:00'), 15752: Timestamp('2015-06-19 19:00:00'), 15799: Timestamp('2015-06-19 15:00:00'), 15899: Timestamp('2015-06-19 20:00:00'), 18077: Timestamp('2015-06-19 21:00:00'), 18405: Timestamp('2015-06-19 23:00:00')}
{15874: Timestamp('2015-06-19 18:47:16.398932'), 17415: Timestamp('2015-06-19 20:45:07.955770'), 16909: Timestamp('2015-06-19 20:02:15.844959'), 15899: Timestamp('2015-06-19 20:00:00'), 15901: Timestamp('2015-06-19 18:40:58.769564'), 15908: Timestamp('2015-06-19 19:08:02.049468'), 17445: Timestamp('2015-06-19 20:46:30.490677'), 15910: Timestamp('2015-06-19 17:22:27.238286'), 16937: Timestamp('2015-06-19 20:02:37.926623'), 15916: Timestamp('2015-06-19 17:20:43.001097'), 17455: Timestamp('2015-06-19 20:5

No node 21096 i nthe leaves stamps
No node 20977 i nthe arrives stamps
No node 20977 i nthe leaves stamps
No node 20948 i nthe arrives stamps
No node 20948 i nthe leaves stamps
No node 20919 i nthe arrives stamps
No node 20919 i nthe leaves stamps
No node 20824 i nthe arrives stamps
No node 20824 i nthe leaves stamps
No node 20949 i nthe arrives stamps
No node 20949 i nthe leaves stamps
No node 20951 i nthe arrives stamps
No node 20951 i nthe leaves stamps
No node 20952 i nthe arrives stamps
No node 20952 i nthe leaves stamps
No node 20946 i nthe arrives stamps
No node 20946 i nthe leaves stamps
No node 20950 i nthe arrives stamps
No node 20950 i nthe leaves stamps
No node 20950 i nthe arrives stamps
No node 20950 i nthe leaves stamps
No node 20957 i nthe arrives stamps
No node 20957 i nthe leaves stamps
No node 20955 i nthe arrives stamps
No node 20955 i nthe leaves stamps
No node 20913 i nthe arrives stamps
No node 20913 i nthe leaves stamps
No node 20898 i nthe arrives stamps
No nod

No node 19478 i nthe arrives stamps
No node 19478 i nthe leaves stamps
No node 19477 i nthe arrives stamps
No node 19477 i nthe leaves stamps
No node 19422 i nthe arrives stamps
No node 19422 i nthe leaves stamps
No node 19419 i nthe arrives stamps
No node 19419 i nthe leaves stamps
No node 19416 i nthe arrives stamps
No node 19416 i nthe leaves stamps
No node 19389 i nthe arrives stamps
No node 19389 i nthe leaves stamps
No node 19384 i nthe arrives stamps
No node 19384 i nthe leaves stamps
No node 19383 i nthe arrives stamps
No node 19383 i nthe leaves stamps
No node 19364 i nthe arrives stamps
No node 19364 i nthe leaves stamps
No node 19363 i nthe arrives stamps
No node 19363 i nthe leaves stamps
No node 19361 i nthe arrives stamps
No node 19361 i nthe leaves stamps
No node 19339 i nthe arrives stamps
No node 19339 i nthe leaves stamps
No node 19335 i nthe arrives stamps
No node 19335 i nthe leaves stamps
No node 19331 i nthe arrives stamps
No node 19331 i nthe leaves stamps
No nod

No node 17070 i nthe arrives stamps
No node 17070 i nthe leaves stamps
No node 17085 i nthe arrives stamps
No node 17085 i nthe leaves stamps
No node 17094 i nthe arrives stamps
No node 17094 i nthe leaves stamps
No node 17107 i nthe arrives stamps
No node 17107 i nthe leaves stamps
No node 17116 i nthe arrives stamps
No node 17116 i nthe leaves stamps
No node 17130 i nthe arrives stamps
No node 17130 i nthe leaves stamps
No node 17142 i nthe arrives stamps
No node 17142 i nthe leaves stamps
No node 17162 i nthe arrives stamps
No node 17162 i nthe leaves stamps
No node 17167 i nthe arrives stamps
No node 17167 i nthe leaves stamps
No node 17183 i nthe arrives stamps
No node 17183 i nthe leaves stamps
No node 17214 i nthe arrives stamps
No node 17214 i nthe leaves stamps
No node 17226 i nthe arrives stamps
No node 17226 i nthe leaves stamps
No node 17238 i nthe arrives stamps
No node 17238 i nthe leaves stamps
No node 17331 i nthe arrives stamps
No node 17331 i nthe leaves stamps
No nod

No node 15916 i nthe arrives stamps
No node 15916 i nthe leaves stamps
No node 15910 i nthe arrives stamps
No node 15910 i nthe leaves stamps
No node 15901 i nthe arrives stamps
No node 15901 i nthe leaves stamps
No node 15874 i nthe arrives stamps
No node 15874 i nthe leaves stamps
No node 15752 i nthe arrives stamps
No node 15752 i nthe leaves stamps
No node 15749 i nthe arrives stamps
No node 15749 i nthe leaves stamps
No node 15745 i nthe arrives stamps
No node 15745 i nthe leaves stamps
No node 15908 i nthe arrives stamps
No node 15908 i nthe leaves stamps
No node 15913 i nthe arrives stamps
No node 15913 i nthe leaves stamps
No node 15924 i nthe arrives stamps
No node 15924 i nthe leaves stamps
No node 15926 i nthe arrives stamps
No node 15926 i nthe leaves stamps
No node 15934 i nthe arrives stamps
No node 15934 i nthe leaves stamps
No node 15945 i nthe arrives stamps
No node 15945 i nthe leaves stamps
No node 15950 i nthe arrives stamps
No node 15950 i nthe leaves stamps
No nod

...

No node 19176 i nthe arrives stamps
No node 19176 i nthe leaves stamps
No node 19179 i nthe arrives stamps
No node 19179 i nthe leaves stamps
No node 19193 i nthe arrives stamps
No node 19193 i nthe leaves stamps
No node 19195 i nthe arrives stamps
No node 19195 i nthe leaves stamps
No node 19197 i nthe arrives stamps
No node 19197 i nthe leaves stamps
No node 19205 i nthe arrives stamps
No node 19205 i nthe leaves stamps
No node 19206 i nthe arrives stamps
No node 19206 i nthe leaves stamps
No node 19208 i nthe arrives stamps
No node 19208 i nthe leaves stamps
No node 19214 i nthe arrives stamps
No node 19214 i nthe leaves stamps
No node 19224 i nthe arrives stamps
No node 19224 i nthe leaves stamps
No node 19227 i nthe arrives stamps
No node 19227 i nthe leaves stamps
No node 19229 i nthe arrives stamps
No node 19229 i nthe leaves stamps
No node 19252 i nthe arrives stamps
No node 19252 i nthe leaves stamps
No node 19253 i nthe arrives stamps
No node 19253 i nthe leaves stamps
No nod

No node 21225 i nthe leaves stamps
No node 21216 i nthe arrives stamps
No node 21216 i nthe leaves stamps
No node 21214 i nthe arrives stamps
No node 21214 i nthe leaves stamps
No node 21207 i nthe arrives stamps
No node 21207 i nthe leaves stamps
No node 21144 i nthe arrives stamps
No node 21144 i nthe leaves stamps
No node 21093 i nthe arrives stamps
No node 21093 i nthe leaves stamps
No node 21100 i nthe arrives stamps
No node 21100 i nthe leaves stamps
No node 21091 i nthe arrives stamps
No node 21091 i nthe leaves stamps
No node 21092 i nthe arrives stamps
No node 21092 i nthe leaves stamps
No node 21090 i nthe arrives stamps
No node 21090 i nthe leaves stamps
No node 21088 i nthe arrives stamps
No node 21088 i nthe leaves stamps
No node 21064 i nthe arrives stamps
No node 21064 i nthe leaves stamps
No node 20631 i nthe arrives stamps
No node 20631 i nthe leaves stamps
No node 20215 i nthe arrives stamps
No node 20215 i nthe leaves stamps
No node 20176 i nthe arrives stamps
No nod

No node 18402 i nthe leaves stamps
No node 18403 i nthe arrives stamps
No node 18403 i nthe leaves stamps
No node 18404 i nthe arrives stamps
No node 18404 i nthe leaves stamps
No node 18408 i nthe arrives stamps
No node 18408 i nthe leaves stamps
No node 18413 i nthe arrives stamps
No node 18413 i nthe leaves stamps
No node 18411 i nthe arrives stamps
No node 18411 i nthe leaves stamps
No node 18410 i nthe arrives stamps
No node 18410 i nthe leaves stamps
No node 18409 i nthe arrives stamps
No node 18409 i nthe leaves stamps
No node 16130 i nthe arrives stamps
No node 16130 i nthe leaves stamps
No node 16128 i nthe arrives stamps
No node 16128 i nthe leaves stamps
No node 16155 i nthe arrives stamps
No node 16155 i nthe leaves stamps
No node 16166 i nthe arrives stamps
No node 16166 i nthe leaves stamps
No node 16163 i nthe arrives stamps
No node 16163 i nthe leaves stamps
No node 16160 i nthe arrives stamps
No node 16160 i nthe leaves stamps
No node 16168 i nthe arrives stamps
No nod

No node 20882 i nthe leaves stamps
No node 20893 i nthe arrives stamps
No node 20893 i nthe leaves stamps
No node 20892 i nthe arrives stamps
No node 20892 i nthe leaves stamps
No node 20890 i nthe arrives stamps
No node 20890 i nthe leaves stamps
No node 20891 i nthe arrives stamps
No node 20891 i nthe leaves stamps
No node 20888 i nthe arrives stamps
No node 20888 i nthe leaves stamps
No node 20881 i nthe arrives stamps
No node 20881 i nthe leaves stamps
No node 20880 i nthe arrives stamps
No node 20880 i nthe leaves stamps
No node 20879 i nthe arrives stamps
No node 20879 i nthe leaves stamps
No node 20874 i nthe arrives stamps
No node 20874 i nthe leaves stamps
No node 20872 i nthe arrives stamps
No node 20872 i nthe leaves stamps
No node 20859 i nthe arrives stamps
No node 20859 i nthe leaves stamps
No node 20844 i nthe arrives stamps
No node 20844 i nthe leaves stamps
No node 20827 i nthe arrives stamps
No node 20827 i nthe leaves stamps
No node 20826 i nthe arrives stamps
No nod

No node 18079 i nthe leaves stamps
No node 18077 i nthe arrives stamps
No node 18077 i nthe leaves stamps
No node 17571 i nthe arrives stamps
No node 17571 i nthe leaves stamps
No node 17455 i nthe arrives stamps
No node 17455 i nthe leaves stamps
No node 17449 i nthe arrives stamps
No node 17449 i nthe leaves stamps
No node 17445 i nthe arrives stamps
No node 17445 i nthe leaves stamps
No node 17415 i nthe arrives stamps
No node 17415 i nthe leaves stamps
No node 17404 i nthe arrives stamps
No node 17404 i nthe leaves stamps
No node 17395 i nthe arrives stamps
No node 17395 i nthe leaves stamps
No node 17357 i nthe arrives stamps
No node 17357 i nthe leaves stamps
No node 17347 i nthe arrives stamps
No node 17347 i nthe leaves stamps
No node 17331 i nthe arrives stamps
No node 17331 i nthe leaves stamps
No node 17238 i nthe arrives stamps
No node 17238 i nthe leaves stamps
No node 17226 i nthe arrives stamps
No node 17226 i nthe leaves stamps
No node 17214 i nthe arrives stamps
No nod

In [57]:
for q in all_nodes[:-1]:
    x = network.nodes.X[q]
    y = network.nodes.Y[q]

In [56]:
for i in all_nodes[-1:
    print i

16130
16128
16155
16166
16163
16160
16168
16170
16169
16171
16156
16149
16134
16125
16123
16117
16113
15916
15910
15901
15874
15752
15749
15745
15908
15913
15924
15926
15934
15945
15950
15959
15963
15967
16049
16054
16058
16105
16107
16112
16158
16181
16190
16288
16599
16622
16639
16909
16937
16979
16988
16996
17070
17085
17094
17107
17116
17130
17142
17162
17167
17183
17214
17226
17238
17331
17347
17357
17395
17404
17415
17445
17449
17455
17571
18077
18079
18084
18310
18311
18313
18333
18402
18403
18404
18408
18413
18411
18410
18409
18406
18414
18405
18401


In [61]:
arrives_tstamps

{18401: Timestamp('2015-06-20 00:00:00'),
 18405: Timestamp('2015-06-19 23:44:28.576885'),
 18406: nan,
 18414: Timestamp('2015-06-19 23:15:31.423115')}